In [1]:
import numpy as np 
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rc('font',family='Times New Roman')

plt.rcParams['text.usetex'] = False

import pandas as pd
import torch

import saferl_algos
from saferl_plotter.logger import SafeLogger
import saferl_utils

import os,sys,argparse,warnings
warnings.filterwarnings("ignore")


source = np.loadtxt('data_dev20.csv')


# source = pd.read_csv('data_6_devs_.csv', sep=' ')
# source = np.array(source)

# source = pd.read_csv('data_0304_mote12_4dev.txt', sep=' ')
# # source = source.drop('date:yyyy-mm-dd', axis = 1)
# source = np.array(source)


import os,sys,argparse,warnings
warnings.filterwarnings("ignore")
import numpy as np
import gym
import torch
import saferl_algos
from saferl_plotter.logger import SafeLogger
import saferl_utils

import skimage.measure 
from sklearn import metrics 


In [3]:
    # open env
    device_num, action_dim = 4, 4

    env = sample_env(source[:,:4], device_num, action_dim, M = 1)
    env._max_episode_steps = 1000    # Done?
    
    state_dim = 3 * env.device_num
    print(f"state_dim:{state_dim}")
    action_dim = env.action_dim
    max_action = env.max_action
 
    kwargs = {
        "state_dim": state_dim,
        "action_dim": action_dim,
        "max_action": max_action,
        "rew_discount": 0.99,
        "tau": 0.005,
        "policy_noise": 0.2* max_action,
        "noise_clip": 0.5 * max_action,
        "policy_freq": 2,
    }

    kwargs_safe = {
        "cost_discount": 0.99,
        "delta": 0.1,                     
    }


state_dim:12


In [4]:
from saferl_algos.fac_sampler import eval_policy
kwargs.update(kwargs_safe)
policy = saferl_algos.fac_sampler.TD3Fac(**kwargs)
# replay_buffer = saferl_utils.CostReplayBuffer(state_dim, action_dim)


# replay_buffer = saferl_utils.MultiTimescaleReplayBuffer(state_dim, action_dim)

Actor state_dim:12
Actor(
  (l1): Linear(in_features=12, out_features=256, bias=True)
  (l2): Linear(in_features=256, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=4, bias=True)
)


In [29]:
policy.load('./models_d4_w_M/dev4_uneven_seed_1_M_2_MTR_1_beta_0.8')

In [30]:
device_num, action_dim = 4, 4

r_weight = np.array([3, 2, 3, 2 ]) # urgency weight
dev_spent_RBs = np.array([1, 1, 1, 1])

M = 1,
cost_weight = 100,

env = sample_env(source[:, :device_num], device_num, action_dim, M, r_weight, dev_spent_RBs, cost_weight)


# x__list = [[] for i in range(env.device_num)]
# aoi_list = [[] for i in range(env.device_num)]
# AoII_list = [[] for i in range(env.device_num)]
instant_comm_load = [[] for i in range(env.device_num)]
# Expected_AoII = [[] for i in range(env.device_num)]
x__list = []
aoi_list = []
AoII_list = []
# instant_comm_load = []
Expected_AoII = []


x__list= np.empty((env.device_num, 0))
aoi_list = np.empty((env.device_num, 0))
AoII_list = np.empty((env.device_num, 0))
tra_data_record = np.empty((1, 0))

state = np.array([0,0,1] * env.device_num)
RB_load = np.empty((1, 0))


sample_time = [0 for i in range(env.device_num)]


for i in range(1000):
    action = policy.select_action(np.array(state))

    for dev in range(env.device_num): 
# #         action = [random.random(), random.random()]
# #         action = np.random.rand(2) 
        if action[dev] > 0:
            sample_time[dev] += 1
        
        instant_comm_load[dev].append(sample_time[dev]/(i+1))

            
    next_state, reward, done, info, = env.step(action)
        
    if i % 100 == 0:  print(f'time:{i}: state:', next_state, f'device_id:{dev}: x_:', env.x_, '\n', f'reward:{reward}')
            
#     x__list += env.x_
    
    x__list =  np.append(x__list, np.array(env.x_).reshape(-1,1), axis =1)
    aoi_list =  np.append(aoi_list, np.array(env.aoi).reshape(-1,1), axis =1)
    AoII_list =  np.append(AoII_list, np.array(env.AoII).reshape(-1,1), axis =1)
    action_cost = action = np.where(action > 0, 1, 0) 
    RB_load =  np.append(RB_load, np.dot(dev_spent_RBs, action).reshape(-1,1), axis =1)
    tra_data_record = np.append(tra_data_record, action[-1].reshape(-1,1), axis =1)
    
#     x__list.append(env.x_)
#     print('X__list: ', x__list,)
#     aoi_list.append(env.aoi)
#     AoII_list.append(env.AoII)


#     if i > 0:
#         Expected_AoII.append( (Expected_AoII[-1]*i + env.AoII)/(i+1) ) 
    state = next_state


# samr040 = instant_comm_load[100:]  # 0.6 constrain    

time:0: state: [2.         0.86847884 1.         2.         0.98220846 1.
 2.         0.86847884 1.         2.         0.98220846 1.        ] device_id:3: x_: [0, 0, 0, 0] 
 reward:-103.0
time:100: state: [0.00000000e+00 3.31727795e-03 1.00000000e+00 0.00000000e+00
 7.67168737e-04 1.00000000e+00 0.00000000e+00 3.31727795e-03
 1.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00] device_id:3: x_: [0.7936031401857822, 1.0433262346859418, 0.7936031401857822, 1.0433262346859418] 
 reward:-0.0720912076756369
time:200: state: [0.00000000e+00 2.84338110e-03 1.00000000e+00 6.00000000e+00
 3.82421992e-03 0.00000000e+00 0.00000000e+00 2.84338110e-03
 1.00000000e+00 0.00000000e+00 1.52968797e-03 1.00000000e+00] device_id:3: x_: [0.7514263204830121, 1.064793010797464, 0.7514263204830121, 1.0686172307133674] 
 reward:-0.21404537500089518
time:300: state: [2.00000000e+00 1.89558740e-03 0.00000000e+00 0.00000000e+00
 3.06635019e-03 1.00000000e+00 4.00000000e+00 1.89558740e-03
 1.00000000e+00 0

In [31]:
print('sample times:', sample_time)
print('tot_RB_cost:', np.dot(dev_spent_RBs, sample_time) )
print('avg_RB_cost:', np.dot(dev_spent_RBs, sample_time) / len(x__list[0]) )
print('tot_RB_cost/M/steps:', np.dot(dev_spent_RBs, sample_time) / env.M / len(x__list[0]) )

# print('RB_cost', np.dot(env.r_weight, sample_time) )
print('avg_aoii:', AoII_list.mean(axis = 1))
w_expected_aoii = np.dot(env.r_weight, AoII_list.mean(axis = 1)) / env.device_num
print('w_expected_aoii:', w_expected_aoii)


nrmse_v = []
for dev in range(device_num):
    nrmse_  = skimage.metrics.normalized_root_mse(source[30:x__list.shape[1], dev],
                                           np.array(x__list[dev, 30:].T) * env.source_mean[dev])
    nrmse_v.append(nrmse_)
b = [float('{:.4f}'.format(i)) for i in nrmse_v]    
print(b)
print(sum(b) / len(b))

sample times: [696, 637, 626, 948]
tot_RB_cost: 2907
avg_RB_cost: 2.907
tot_RB_cost/M/steps: [2.907]
avg_aoii: [0.062 0.213 0.103 0.07 ]
w_expected_aoii: 0.026525
[0.0042, 0.01, 0.0044, 0.0048]
0.00585


(array([1, 1, 1, 1]),)

In [13]:
class sample_env():
    def __init__(
        self,
        source,
        device_num,
        action_dim=1,
        M = 30,
        
        # r_weight = np.array([3, 2, 3, 2, 1, 1]), # urgency weight
        # dev_spent_RBs = np.array([1, 1, 1, 1, 5, 5]), 

        r_weight = np.array([5, 2, 5, 2]), # urgency weight
        dev_spent_RBs = np.array([1, 1, 1, 1 ]), 

        cost_weight = int(5e1),
        max_action=1,
        delta=1,

        rew_discount=0.99,
        cost_discount=0.99,
        tau=0.005,
        policy_noise=0.2,
        noise_clip=0.5,
        policy_freq=2,
        delay = 2,
        prob_pass = 0.75,
        ksi = 0.0001
    ):


        self.device_num = device_num
        self.action_dim = action_dim
        self.max_action = max_action

        self._max_episode_steps = int(9e3)
        self.tau  = tau
        
        self.prob_pass = prob_pass
        self.delay = delay
        self.ksi = ksi
        self.t = 0
        self.aoi = [1 for i in range(self.device_num)]
        self.x_ = [0 for i in range(self.device_num)]
        self.AoII = np.array([1 for i in range(self.device_num)])
        self.arrive_times =[[] for i in range(self.device_num)]
        self.send_times = [[] for i in range(self.device_num)]
        self.last_recv_t = [0 for i in range(self.device_num)]
        self.source_mean = source.mean(axis=0)
        self.source = source/self.source_mean
        self.error_indicator = [False for i in range(self.device_num)] 


        self.r_weight = np.array(r_weight) / np.sum(r_weight) 
        self.dev_spent_RBs = dev_spent_RBs
        self.M = M
        self.cost_weight = cost_weight

        self.done = False
        self.reset_next_state_as = [[] for i in range(self.device_num)]

        
        
    def step(self, action = [0,0]):
        all_dev_state = [[] for i in range(self.device_num)]
        for dev in range(self.device_num):        
            x = self.source[self.t, dev]
            if action[dev] > 0:
                self.send_times[dev].append(self.t)
                self.arrive_times[dev].append(self.t + self.delay)

            # loss pkg?     
            pkg_lost = np.random.uniform(0, 1) >= self.prob_pass

            gamma_t = 0 if pkg_lost else 1 
            
            error = abs(self.x_[dev] - x) 

            if dev < 16 and dev % 2 == 0:
                self.error_indicator[dev] = error / x > 1e-2     # temp   1e-1  4e-2
            if dev < 16 and dev % 2 == 1:
                self.error_indicator[dev] = error / x >= 1e-2   # humi   1e-1    4e-2
            elif dev >= 16:
                self.error_indicator[dev] = error > 5e-2         # traj  1e-1   3e-1

            # self.error_indicator[dev] = error >= self.ksi 

            # AoI evoluation, only when pkg is received   
            if pkg_lost == False and self.t in self.arrive_times[dev]:
                self.aoi[dev] = self.t - self.send_times[dev][-1] # update aoi
                # del self.send_times[0]
                self.last_recv_t[dev] = self.t
                self.x_[dev] = x # update estimation result
            else: 
                self.aoi[dev] += 1

            self.AoII[dev] = self.aoi[dev] * (1 if self.error_indicator[dev] else 0)

            if self.t in self.arrive_times[dev]:
                self.arrive_times[dev].remove(self.t)
            if self.t - self.delay in self.send_times[dev]:
                self.send_times[dev].remove(self.t - self.delay)
            
            dev_state = np.array([self.aoi[dev], error,  gamma_t])
            all_dev_state[dev] = dev_state
        self.t += 1


        next_state = 1 
        reward = 1
        self.done =  0 if self.t < self._max_episode_steps  else 1
        info = "step"

        if self.t < self._max_episode_steps:
            pass
        else:
            self.reset_next_state_as = np.array(all_dev_state).reshape(3 * self.device_num)

        RB_cost = np.dot(dev_spent_RBs.T, action)
        RB_cost = RB_cost - self.M if RB_cost > self.M else 0
        return  np.array(all_dev_state).reshape(3 * self.device_num), \
                -100 * skimage.metrics.normalized_root_mse(source[self.t, : self.device_num], \
                    np.array(self.x_) * env.source_mean[:])  -5 * np.max(env.AoII * env.r_weight) , \
                self.done, info
                # np.dot( self.r_weight ,(-1) * self.AoII.reshape(self.device_num) ), \
                # (-1 * np.max(env.AoII * env.r_weight) ), \
                # -1000 * skimage.metrics.normalized_root_mse(source[self.t, :], \
                #     np.array(self.x_) * env.source_mean[:]) + (-1 * np.max(env.AoII * env.r_weight) ),  \


                
        # np.array([self.aoi, error,  gamma_t]), -self.AoII, done, info 
        # return next_state, reward , done, info, pkg_lost, AoII, self.t, self.arrive_times, self.send_times, self.aoi, self.x_
    
    def reset(self):
        self.t = 0 
        self.arrive_times =[[] for i in range(self.device_num)]
        self.send_times = [[] for i in range(self.device_num)]
        if self.done:
            reset_state = self.reset_next_state_as
        else: reset_state = np.array([-1, 0, 0]*self.device_num)
        return [reset_state, self.t]

